# Coursera - Capstone Project for IBM Data Science Certificate
## "The battle of the neighborhoods" 

**Description**

This is the final assignment the Coursera Capstone Project
> https://www.coursera.org/learn/applied-data-science-capstone/home/info

**Objective:** Recommend a city in Dubai best fit for opening a new Hotel / Restaurant.


**Data Sources:** (csv file of Dubai Neighborhood coordinates via Box)

> https://app.box.com/s/gv2vdfvgsa4c6r23k6ia2ikgso14yrkz

**GitHub repository**

> https://github.com/tookitooki12/Coursera_Capstone

**Table of contents:**

*   **System & Data Setup**
*   **1** - Create table with postal codes
*   **2** - Setup Neighborhood map using folium & 'Latititude', 'Longitude' from csv
*   **3** - Cluster Venues by neighborhood and analysis of 'best' fit for new location

## System & Data Setup

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#mapping tools
!pip install geopy 
from geopy.geocoders import Nominatim 

!pip install folium
import folium # map rendering library

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

**be sure to load 'Dubai_neighborhoods.csv' into your working directory**

In [2]:
# read csv file once loaded into working directory listed above
Geospacial_Coordinates = pd.read_csv('https://app.box.com/shared/static/gv2vdfvgsa4c6r23k6ia2ikgso14yrkz.csv', sep = ',') 
# examine the shape of original input data
print(Geospacial_Coordinates.shape)

(23, 8)


## Part 2 - Setup Dubai Neighborhood Map

In [3]:
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [4]:
Geo = pd.DataFrame(Geospacial_Coordinates)
Geo.head()

,Neighborhood,Avg Rent Per Unit,Z-Score,Distance from Palm,Distance from Zabeel,Distance from Jumeirah,Latitude,Longitude
0,Discovery Gardens,"44,672",-1.53,8.18,26.15,20.73,25.0390,55.1445
1,Dubai Silicon Oasis,"54,417",-1.30,24.96,13.31,16.39,25.1279,55.3863
2,Jumeirah Village Circle,"60,068",-1.17,9.16,20.56,16.13,25.0602,55.2094
3,Dubai Sports City,"62,753",-1.10,11.36,22.32,18.28,25.0391,55.2176
4,Remraam,"67,284",-0.99,16.71,25.27,22.27,25.0014,55.2508


In [5]:
Geo.dtypes

Neighborhood               object
Avg Rent Per Unit          object
Z-Score                   float64
Distance from Palm        float64
Distance from Zabeel      float64
Distance from Jumeirah    float64
Latitude                  float64
Longitude                 float64
dtype: object

In [6]:
address = 'Dubai, UAE'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dubai, UAE are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dubai, UAE are 25.0750095, 55.1887608818332.


In [7]:
# create map of Dubai using latitude and longitude values
map_dubai = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(Geo['Latitude'], Geo['Longitude'], Geo['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dubai)  
    
map_dubai

### Part 2a - initial neighborhood comparison using Foursquare API

In [8]:
CLIENT_ID = 'AR2WMOIEQ2H1CLUCD5FPYJLWOQOOEQ2SNYYUIJQS5NKFVDV1' # my Foursquare ID
CLIENT_SECRET = 'P3PCXVFK5CLFBLE25KUXVHQ0YNUNTV4KGGBSSXQ4AEDQLQ3E' # your Foursquare Secret
VERSION = '20190303' # Foursquare API version
radius = 500
LIMIT = 250

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AR2WMOIEQ2H1CLUCD5FPYJLWOQOOEQ2SNYYUIJQS5NKFVDV1
CLIENT_SECRET:P3PCXVFK5CLFBLE25KUXVHQ0YNUNTV4KGGBSSXQ4AEDQLQ3E


**Let's explore 'Dubai Investment Park' which sounds wealthy**

In [9]:
#define objects for 'Studio District' index [15] in Geo
neighborhood_latitude = Geo.loc[16, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[16, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[16, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Dubai Investments Park are 24.979, 55.1762.


**Now, let's get the top 100 venues that are in Dubai Investment Park within a radius of 500 meters.**

In [10]:
#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display GET request URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AR2WMOIEQ2H1CLUCD5FPYJLWOQOOEQ2SNYYUIJQS5NKFVDV1&client_secret=P3PCXVFK5CLFBLE25KUXVHQ0YNUNTV4KGGBSSXQ4AEDQLQ3E&v=20190303&ll=24.979,55.1762&radius=500&limit=250'

In [11]:
results = requests.get(url).json()
results # remove ';' if you want to see json data

{'meta': {'code': 200, 'requestId': '5c7e0927f594df76b8f68194'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 24.983500004500005,
    'lng': 55.181155088448826},
   'sw': {'lat': 24.974499995499993, 'lng': 55.17124491155118}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5214eb2611d2556daaa1461d',
       'name': 'Rock Republic',
       'location': {'address': 'Dubai Investment Park',
        'lat': 24.980327525686636,
        'lng': 55.17418454504484,
        'labeledLatLngs': [{'la

**clean the json and structure it into a *pandas* dataframe.**

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
df_IP = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_IP = df_IP.loc[:, filtered_columns]

# filter the category for each row
df_IP['venue.categories'] = df_IP.apply(get_category_type, axis=1)

# clean columns

df_IP.columns = [col.split(".")[-1] for col in df_IP.columns]
df_IP.insert(0, 'neighborhood', 'Dubai Investment Park')

print('{} venues were returned by Foursquare.'.format(df_IP.shape[0]))
df_IP.head()

3 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,Dubai Investment Park,Rock Republic,Rock Climbing Spot,24.980328,55.174185
1,Dubai Investment Park,CanvasJet,Print Shop,24.979171,55.176076
2,Dubai Investment Park,Fetchr Warehouse,Shipping Store,24.976082,55.174012


**create a map of the Marina district and highlight nearby venues

In [14]:
map_IP = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)

# add markers to map
for lat, lng, name, categories in zip(df_IP['lat'], df_IP['lng'], df_IP['name'], df_IP['categories']):
  label = '{},{}'.format(categories,name)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7).add_to(map_IP) 
    
map_IP

### Let's create a similar dataframe for each neighborhood: 

**index # 9 - Al Sufouh**

In [15]:
#define objects for 'Al Soufouh' index [9] in Geo
neighborhood_latitude = Geo.loc[9, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[9, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[9, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_ASufouh = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_ASufouh = df_ASufouh.loc[:, filtered_columns]

# filter the category for each row
df_ASufouh['venue.categories'] = df_ASufouh.apply(get_category_type, axis=1)

# clean columns

df_ASufouh.columns = [col.split(".")[-1] for col in df_ASufouh.columns]
df_ASufouh.insert(0, 'neighborhood', 'Al Sufouh')

print('{} venues were returned by Foursquare.'.format(df_ASufouh.shape[0]))
df_ASufouh.head()

5 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,Al Sufouh,Emirates Co-Op,Grocery Store,25.112435,55.173827
1,Al Sufouh,Shaikhath Al Arab Cafeteria شيخة العرب كافتيريا,Cafeteria,25.112604,55.173924
2,Al Sufouh,Marina Pharmacy,Pharmacy,25.112530,55.173813
3,Al Sufouh,Al Sufouh Park,Playground,25.113393,55.171660
4,Al Sufouh,Pearl Residence Gym,College Gym,25.110788,55.179856


**index # 10 DIFC**

In [16]:
#define objects for 'DIFC' index [10] in Geo
neighborhood_latitude = Geo.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[10, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[10, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_DIFC = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_DIFC = df_DIFC.loc[:, filtered_columns]

# filter the category for each row
df_DIFC['venue.categories'] = df_DIFC.apply(get_category_type, axis=1)

# clean columns

df_DIFC.columns = [col.split(".")[-1] for col in df_DIFC.columns]
df_DIFC.insert(0, 'neighborhood', 'DIFC')

print('{} venues were returned by Foursquare.'.format(df_DIFC.shape[0]))
df_DIFC.head()

79 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,DIFC,The Ritz-Carlton,Hotel,25.212517,55.279399
1,DIFC,The Sunken Garden,Hookah Bar,25.212093,55.280039
2,DIFC,Burger & Lobster,Seafood Restaurant,25.211287,55.281787
3,DIFC,Carnival by Trèsind,Indian Restaurant,25.211010,55.282113
4,DIFC,Café Belge,Belgian Restaurant,25.212062,55.279914


**index # 11 Business Bay**

In [17]:
#define objects for 'Business_Bay' index [11] in Geo
neighborhood_latitude = Geo.loc[11, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[11, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[11, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_Business_Bay = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_Business_Bay = df_Business_Bay.loc[:, filtered_columns]

# filter the category for each row
df_Business_Bay['venue.categories'] = df_Business_Bay.apply(get_category_type, axis=1)

# clean columns

df_Business_Bay.columns = [col.split(".")[-1] for col in df_Business_Bay.columns]
df_Business_Bay.insert(0, 'neighborhood', 'Business Bay')

print('{} venues were returned by Foursquare.'.format(df_Business_Bay.shape[0]))
df_Business_Bay.head()

23 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,Business Bay,"Renaissance Downtown Hotel, Dubai",Hotel,25.185675,55.273650
1,Business Bay,Basta!,Italian Restaurant,25.185750,55.273635
2,Business Bay,BHAR,Middle Eastern Restaurant,25.185598,55.273534
3,Business Bay,Six Senses Spa Dubai,Spa,25.185825,55.273563
4,Business Bay,Gulf Court Hotel Business Bay,Hotel,25.182194,55.274875


**index # 12 Jumeirah Lakes Towers**

In [18]:
#define objects for 'JLT' index [12] in Geo
neighborhood_latitude = Geo.loc[12, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[12, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[12, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_JLT = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_JLT = df_JLT.loc[:, filtered_columns]

# filter the category for each row
df_JLT['venue.categories'] = df_JLT.apply(get_category_type, axis=1)

# clean columns

df_JLT.columns = [col.split(".")[-1] for col in df_JLT.columns]
df_JLT.insert(0, 'neighborhood', 'Jumeirah Lakes Towers')

print('{} venues were returned by Foursquare.'.format(df_JLT.shape[0]))
df_JLT.head()

36 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,Jumeirah Lakes Towers,Bait Maryam,Theme Restaurant,25.070765,55.141889
1,Jumeirah Lakes Towers,Wokyo Noodle Bar,Noodle House,25.068151,55.140931
2,Jumeirah Lakes Towers,Betawi,Indonesian Restaurant,25.069993,55.141876
3,Jumeirah Lakes Towers,Golositalia,Italian Restaurant,25.069462,55.140268
4,Jumeirah Lakes Towers,Fidelity Fitness Club,Gym,25.068784,55.141660


**index 13 Barsha Heights**

In [19]:
#define objects for 'Barsha' index [12] in Geo
neighborhood_latitude = Geo.loc[13, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[13, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[13, 'Neighborhood'] # neighborhood name

#step 1 - create the correct GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
df_Barsha = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
df_Barsha = df_Barsha.loc[:, filtered_columns]

# filter the category for each row
df_Barsha['venue.categories'] = df_Barsha.apply(get_category_type, axis=1)

# clean columns

df_Barsha.columns = [col.split(".")[-1] for col in df_Barsha.columns]
df_Barsha.insert(0, 'neighborhood', 'Barsha Heights')

print('{} venues were returned by Foursquare.'.format(df_Barsha.shape[0]))
df_Barsha.head()

60 venues were returned by Foursquare.


,neighborhood,name,categories,lat,lng
0,Barsha Heights,AURIS INN Al MUHANNA Hotel,Hotel,25.094750,55.177058
1,Barsha Heights,Beef King,Chinese Restaurant,25.096673,55.175715
2,Barsha Heights,TRYP by Wyndham Dubai,Hotel,25.097234,55.174834
3,Barsha Heights,MMA Fitness Center,Gym / Fitness Center,25.096647,55.175727
4,Barsha Heights,Fuchsia,Thai Restaurant,25.095363,55.178584


**analysis of venue distribution**

In [20]:
df_venues = pd.concat([df_Barsha, df_JLT, df_Business_Bay, df_DIFC, df_ASufouh])
df_venues['count'] = 1
df_venues.shape

(203, 6)

In [21]:
total_venues = pd.pivot_table(df_venues,index=["neighborhood"], values=["count"],aggfunc=np.sum)
total_venues

,count
neighborhood,
Al Sufouh,5
Barsha Heights,60
Business Bay,23
DIFC,79
Jumeirah Lakes Towers,36


In [22]:
df_venues2 = df_venues.copy()
df_venues3 = df_venues.copy()
df_venues_rest = df_venues2[df_venues2['categories'].str.contains('Restaurant')].reset_index(drop=True)
df_venues_rest['Venue Type'] = 'Restaurant'
df_venues_hotel = df_venues3[df_venues3['categories'].str.contains('Hotel')].reset_index(drop=True)
df_venues_hotel['Venue Type'] = 'Hotel'
df_venues_final = pd.concat([df_venues_rest,df_venues_hotel]).reset_index(drop=True)
df_venues_final.shape

(94, 7)

In [23]:
pivot = pd.pivot_table(df_venues_final,index=["neighborhood","Venue Type"], values=["count"],aggfunc=np.sum)
pivot

count
neighborhood          Venue Type       
Barsha Heights        Hotel          14
                      Restaurant     17
Business Bay          Hotel           4
                      Restaurant     12
DIFC                  Hotel           6
                      Restaurant     27
Jumeirah Lakes Towers Hotel           1
                      Restaurant     13

In [24]:
box = df_venues_final.groupby('neighborhood')['Venue Type']\
    .value_counts()\
    .unstack(level=1)\
    .plot.bar(stacked=True)
box

**create 'one hot' file with dummy values by venue category**

In [25]:
# one hot encoding
dubai_onehot = pd.get_dummies(df_venues_final[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dubai_onehot['neighborhood'] = df_venues_final['neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dubai_onehot.columns[-1]] + list(dubai_onehot.columns[:-1])
dubai_onehot = dubai_onehot[fixed_columns]

dubai_onehot.head()

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Chinese Restaurant,Comfort Food Restaurant,English Restaurant,French Restaurant,German Restaurant,...,Molecular Gastronomy Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,Spanish Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vietnamese Restaurant
0,Barsha Heights,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Barsha Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,Barsha Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Barsha Heights,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barsha Heights,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [26]:
dubai_grouped = dubai_onehot.groupby('neighborhood').mean().reset_index()
dubai_grouped

,neighborhood,African Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Chinese Restaurant,Comfort Food Restaurant,English Restaurant,French Restaurant,German Restaurant,...,Molecular Gastronomy Restaurant,Peruvian Restaurant,Restaurant,Seafood Restaurant,Spanish Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Vietnamese Restaurant
0,Barsha Heights,0.000000,0.000000,0.000000,0.000000,0.032258,0.032258,0.000000,0.032258,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.064516,0.000000,0.032258,0.000000
1,Business Bay,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.062500,0.0625,...,0.000000,0.000000,0.187500,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000
2,DIFC,0.030303,0.030303,0.060606,0.030303,0.030303,0.000000,0.030303,0.030303,0.0000,...,0.000000,0.030303,0.121212,0.030303,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
3,Jumeirah Lakes Towers,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.071429,0.000000,0.000000,0.000000,0.071429,0.0000,0.071429,0.071429,0.000000,0.142857


In [27]:
dubai_grouped.shape

(4, 33)

**Let's print each neighborhood along with the top 15 most common venues**

In [28]:
num_top_venues = 15

for hood in dubai_grouped['neighborhood']:
    print("----"+hood+"----")
    temp = dubai_grouped[dubai_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barsha Heights----
                         venue  freq
0                        Hotel  0.42
1    Middle Eastern Restaurant  0.19
2              Thai Restaurant  0.06
3           Italian Restaurant  0.06
4           Turkish Restaurant  0.03
5            Indian Restaurant  0.03
6           Mexican Restaurant  0.03
7          Lebanese Restaurant  0.03
8                    Hotel Bar  0.03
9            French Restaurant  0.03
10     Comfort Food Restaurant  0.03
11          Chinese Restaurant  0.03
12          Spanish Restaurant  0.00
13  Modern European Restaurant  0.00
14            Theme Restaurant  0.00


----Business Bay----
                              venue  freq
0                             Hotel  0.25
1                        Restaurant  0.19
2         Middle Eastern Restaurant  0.12
3                Italian Restaurant  0.12
4                Chinese Restaurant  0.06
5               Japanese Restaurant  0.06
6                  Tapas Restaurant  0.06
7                 French R

**First, let's write a function to sort the venues in descending order.**

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = dubai_grouped['neighborhood']

for ind in np.arange(dubai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dubai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barsha Heights,Hotel,Middle Eastern Restaurant,Italian Restaurant,Thai Restaurant,French Restaurant
1,Business Bay,Hotel,Restaurant,Italian Restaurant,Middle Eastern Restaurant,German Restaurant
2,DIFC,Hotel,Italian Restaurant,Restaurant,Indian Restaurant,Middle Eastern Restaurant
3,Jumeirah Lakes Towers,Vietnamese Restaurant,Italian Restaurant,Molecular Gastronomy Restaurant,Indonesian Restaurant,Greek Restaurant


In [31]:
#define objects for 'DIFC' index [10] in Geo
neighborhood_latitude = Geo.loc[10, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Geo.loc[10, 'Longitude'] # neighborhood longitude value
neighborhood_name = Geo.loc[10, 'Neighborhood'] # neighborhood name

map_DIFC = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)

# add markers to map
for lat, lng, name, categories in zip(df_DIFC['lat'], df_DIFC['lng'], df_DIFC['name'], df_DIFC['categories']):
  label = '{},{}'.format(categories,name)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7).add_to(map_DIFC) 
    
map_DIFC